In [2]:
import os
import time
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool as ThreadPool
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from functools import partial
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By    
from selenium.webdriver.support import expected_conditions as EC

In [65]:
def element_exits(xpath):
    
    try:
        driver.find_element_by_xpath(xpath)

        return True

    except:

        return False

In [2]:
pro_list = pd.read_excel('./Pro_list.xlsx')
web_list = pro_list['Website for google scholar']
web_list = web_list.to_list()
prof_name_list = pro_list['Name'].to_list()

In [7]:
######追加新的信息
import pandas as pd
prof_list_new = pd.read_csv('Pro_list.xlsx')
pro_info = pd.read_csv('prof_info.csv',header=None)
prof_list_new = prof_list_new[~prof_list_new.Name.isin(pro_info[0])]
web_list, prof_name_list = prof_list_new['Website for google scholar'].to_list(),  prof_list_new['Name'].to_list()


In [11]:
list(prof_list_new['Website for google scholar'])

['https://scholar.google.com/citations?hl=en&user=hR249csAAAAJ',
 'https://scholar.google.com/citations?hl=en&user=luSyn1AAAAAJ',
 'https://scholar.google.com/citations?user=FABZCeAAAAAJ&hl=en&oi=ao',
 'https://scholar.google.com/citations?user=9560QjYAAAAJ&hl=en&oi=ao',
 'https://scholar.google.com.sg/citations?user=tCxawzgAAAAJ&hl=zh-CN',
 'https://scholar.google.com.sg/citations?user=EEIegd4AAAAJ',
 'https://scholar.google.com.sg/citations?user=Ze39iM0AAAAJ',
 'https://scholar.google.com.sg/citations?user=q6ljRLoAAAAJ&hl=en',
 'https://scholar.google.com.sg/citations?user=z8_x7C8AAAAJ',
 'https://scholar.google.com/citations?hl=en&user=UozjxW8AAAAJ',
 'https://scholar.google.com/citations?hl=en&user=6Y-m8k4AAAAJ',
 'https://scholar.google.com/citations?hl=en&user=30Fp0YYAAAAJ',
 'https://scholar.google.com/citations?hl=en&user=Ton5pYMAAAAJ',
 'https://scholar.google.com/citations?hl=en&user=F_YjttQAAAAJ',
 'https://scholar.google.com.sg/citations?user=Kot7RHQAAAAJ',
 'https://schola

In [29]:
def crawler(ind):
    web,prof_name = web_list[ind], prof_name_list[ind]
    paper_list = []
    citation_list = []
    year_list = []
    author_list = []
    discription_list = []
    
    
    chrome_options = Options()
    chrome_options.add_argument("--headless") 
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.maximize_window()
    driver.get(web)
    
    print('Crawling google scholar website of Porf:{}'.format(prof_name))
    
    research_area = driver.find_element_by_xpath('//div[@id="gsc_prf_int"]').text
    print(research_area)
    if not research_area:
        research_area = ' '
    try:
        
        time.sleep(0.5)
        #year
        for year in driver.find_elements_by_xpath('//span[@class="gsc_a_h gsc_a_hc gs_ibl"]'):
            
            year_list.append(year.text)
        
        #citation
        for citation in driver.find_elements_by_xpath('//a[@class="gsc_a_ac gs_ibl"]'):
            #print(citation.text)
            citation_list.append(citation.text)
       
        
        #paper author discription   
        for each_element in driver.find_elements_by_xpath('//a[@class="gsc_a_at"]'):
            #print(each_element.text)
            paper_list.append(each_element.text)
            
            python_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="gsc_a_b"]/tr[1]/td[1]/a')))
            driver.execute_script("arguments[0].scrollIntoView();", each_element)
            driver.execute_script("arguments[0].click();", each_element)
            
            
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@id="gsc_vcd_table"]')))
            try:
                author = driver.find_element_by_xpath('//div[contains(text(),"Authors")]/following::*').text
            except:
                author = prof_name
            author_list.append(author)
         
            try:
                discription = driver.find_element_by_xpath('//*[@id="gsc_vcd_descr"]/div').text
                
            except:
                discription = 'None'
            discription_list.append(discription)
            #print(discription)
              
            close_button = driver.find_element_by_xpath('//*[@id="gs_md_cita-d-x"]/span[1]')
            driver.execute_script("arguments[0].scrollIntoView();", close_button)
            driver.execute_script("arguments[0].click();", close_button)
            time.sleep(1)
        while len(citation_list)<len(year_list):
            
            citation_list.append(citation_list[-1])
        
        if len(year_list) == len(citation_list) and len(paper_list)==len(year_list):
            temp_df = pd.DataFrame(
            {
                'Name': [prof_name]*len(year_list),
                'Google_Link' :[web]*len(year_list),
                'Paper': paper_list,
                'Author': author_list,
                'Citation': citation_list,
                'Year': year_list,
                'Discription': discription_list
            }
            )
            temp_df.to_csv('./prof_info.csv',index = False, mode= 'a',header= False)
            print('Successfully Crawling!')
        else:
            print('长度错了')
            print('Crawling Failure!(Store failure info)')

    except:
            print('元素抓取报错')
            print('Crawling Failure!(Store failure info)')                                                                             
    driver.quit()                                
                                                                                
                                                                                

In [30]:
pool = ThreadPool(processes=4)
pool.map(crawler,list(range(len(web_list))))
pool.close()
pool.join()

Crawling google scholar website of Porf:NG Hwee Tou
Natural Language ProcessingComputational Linguistics
Crawling google scholar website of Porf:Zhi Ning Chen
electromagnetic engineering
Crawling google scholar website of Porf:HANG Chang Chieh
innovation strategy & managementintellectual property & innovationframework for analyzing disruptive technologies
Crawling google scholar website of Porf:Shafiq Joty
Natural Language ProcessingMachine Learning
Successfully Crawling!
Successfully Crawling!
Successfully Crawling!
Successfully Crawling!
Crawling google scholar website of Porf:Chengkuo Lee
MEMSEnergy HarvestingNanophotonicsSensorMetamaterials
Crawling google scholar website of Porf:Xiao Gong
Semiconductor DevicesRF devices and circuitsFlexible Electronic SystemsPhotonic Devices
Crawling google scholar website of Porf:Diptarka CHAKRABORTY
Theoretical Computer Science
Crawling google scholar website of Porf:OOI Beng Chin
Databasesdistributed systemsdata analytics
Successfully Crawling!